In [6]:
import pandas as pd
from delta import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from heroes_pipeline.utils import get_spark_session


In [7]:
spark = get_spark_session()


23/07/26 14:36:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
# Sample data for source DataFrame
source_data = {
    'ID': [1, 2, 3],
    'Value': [100, 200, 300]
}
source_df = pd.DataFrame(source_data)

# Sample data for target DataFrame
target_data = {
    'ID': [1, 2, 4],
    'Value': [150, 250, 400]
}
target_df = pd.DataFrame(target_data)

In [5]:
source_df.head()

,ID,Value
0,1,100
1,2,200
2,3,300


In [6]:
target_df.head()

,ID,Value
0,1,150
1,2,250
2,4,400


In [9]:
# Convert the pandas DataFrames to Spark DataFrames and save as Delta tables
source_df_spark = spark.createDataFrame(source_df)
target_df_spark = spark.createDataFrame(target_df)

source_df_spark.write.format("delta").mode("overwrite").save("/delta/source_table")
target_df_spark.write.format("delta").mode("overwrite").save("/delta/target_table")

Py4JJavaError: An error occurred while calling o127.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:738)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:860)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:256)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:239)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Delegati

In [ ]:

# Define the merge condition
merge_condition = "source_table.ID = target_table.ID"

# Perform the merge operation
delta_table = DeltaTable.forPath(spark, "/delta/target_table")
delta_table.alias("target_table").merge(
    source=spark.table("source_table").alias("source_table"),
    condition=merge_condition
).whenMatchedUpdate(set={
    "Value": col("source_table.Value")  # Update the "Value" column with the value from source_df
}).whenNotMatchedInsert(
    values={
        "ID": col("source_table.ID"),     # Insert the "ID" from source_df
        "Value": col("source_table.Value") # Insert the "Value" from source_df
    }
).execute()



In [ ]:
# Display the updated target table after the merge operation
result_df = spark.read.format("delta").load("/delta/target_table")
result_df.show()